In [712]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.model_selection import train_test_split

In [3]:
import json
from itertools import islice

ratings, book_features = get_data()

In [386]:
# Need to edit this to where your csvs are.
reviews = pd.read_csv('face_reviews.csv').append(pd.read_csv('eye_reviews.csv'), ignore_index=True).append(pd.read_csv('cheek_reviews.csv'), ignore_index=True).append(pd.read_csv('lip_reviews.csv'), ignore_index=True)


In [688]:
dataset = Dataset()
dataset.fit(users = (x for x in face_reviews.userID),
            items = (x for x in face_reviews.productID))

In [689]:
len(face_reviews)

415197

In [690]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 232600, num_items 1976.


In [691]:
(interactions, weights) = dataset.build_interactions(list(zip(reviews.userID, reviews.productID)))
print(repr(train_interactions))

<232600x1976 sparse matrix of type '<class 'numpy.float32'>'
	with 373677 stored elements in COOrdinate format>


In [692]:
from lightfm.cross_validation import random_train_test_split

In [693]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.1, random_state = np.random.RandomState(42))

In [694]:
NUM_THREADS = 2
NUM_COMPONENTS = 20
NUM_EPOCHS = 5
ITEM_ALPHA = 1e-6

In [695]:
NUM_THREADS = 2
NUM_COMPONENTS = 30
NUM_EPOCHS = 5
ITEM_ALPHA = 1e-6

In [696]:
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

In [697]:
%time model = model.fit(train_interactions, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

CPU times: user 3.16 s, sys: 36.1 ms, total: 3.19 s
Wall time: 3.19 s


In [698]:
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train_interactions, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Collaborative filtering train AUC: 0.9862645


In [699]:
model.item_biases *= 0.0
test_auc = auc_score(model, test_interactions, num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

Collaborative filtering test AUC: 0.661877


In [700]:
# Found this on the internet, don't know if we need to incorporate it.
def sample_recommendation(model, data, user_ids):


    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [714]:
predict = model.predict(user_ids=0, item_ids=list(item_id_map.values()))
predict

array([ 2.88989282, -0.07361358, -0.69099897, ..., -0.25805008,
       -1.25469661, -0.60276353])